use python 3.9.13


In [ ]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# pip install -r ../requirements.txt

# Rancangan


---

[ Pilih Soal ]: ⬇️ Stimulus A / Stimulus B

[ Stimulus ]:
Pemanasan global terjadi karena...

[ Pertanyaan ]:
Apa yang menjadi tantangan...

[ Jawaban Siswa ]:
✎ **\*\*\*\***\_**\*\*\*\***--

[🔍 Nilai Jawaban]

📊 Hasil:
✅ Jawaban diklasifikasikan sebagai "Relevan dan Benar" (Label: 1)

🧠 Alasan: Jawaban mengandung kata kunci "pindah", "ekonomi", "sosial"


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset

import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report,f1_score
import numpy as np

In [ ]:
from sentence_transformers import SentenceTransformer # type: ignore


In [ ]:
import optuna
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

---


# Data Understanding


In [ ]:
train_a = pd.read_csv('../Dataset/Data_A/data_train_A.csv')
dev_a = pd.read_csv('../Dataset/Data_A/data_dev_A.csv')
test_a = pd.read_csv('../Dataset/Data_A/data_test_A.csv')
train_b = pd.read_csv('../Dataset/Data_B/data_train_B.csv')
dev_b = pd.read_csv('../Dataset/Data_B/data_dev_B.csv')
test_b = pd.read_csv('../Dataset/Data_B/data_test_B.csv')

In [ ]:
print("Data Train A:")
print(train_a.head())
print("\nKolom:", train_a.columns.tolist())

In [ ]:
test_a

In [ ]:
print("Data Train B:")
print(train_b.head())
print("\nKolom:", train_b.columns.tolist())

In [ ]:
stimulus_a = ["Pemanasan global terjadi karena peningkatan produksi karbon dioksida yang dihasilkan oleh pembakaran fosil dan konsumsi bahan bakar yang tinggi.",
"Salah satu akibat adalah mencairnya es abadi di kutub utara dan selatan yang menimbulkan naiknya ketinggian air laut.",
"kenaikan air laut akan terjadi terus menerus meskipun dalam hitungan centimeter akan mengakibatkan perubahan yang signifikan.",
"Film “Waterworld”, adalah film fiksi ilmiah yang menunjukkan akibat adanya pemanasan global yang sangat besar sehingga menyebabkan bumi menjadi tertutup oleh lautan.",
"Negara-negara dan daratan yang dulunya kering menjadi tengelamn karena terjadi kenaikan permukaan air laut.",
"Penduduk yang dulunya bisa berkehidupan bebas menjadi terpaksa mengungsi ke daratan yang lebih tinggi atau tinggal diatas air.",
"Apa yang akan menjadi tantangan bagi suatu penduduk ketika terjadi situasi daratan tidak dapat ditinggali kembali karena tengelam oleh naiknya air laut."]

stimulus_b = ["Sebuah toko baju berkonsep self-service menawarkan promosi dua buah baju bertema tahun baru seharga Rp50.000,00. sebelum baju bertema tahun baru dibagikan kepada pembeli, sebuah layar akan menampilkan tampilan gambar yang menampilkan kondisi kerja di dalam sebuah pabrik konveksi/pembuatan baju. ",
"Kemudian pembeli diberi program pilihan untuk menyelesaikan pembeliannya atau menyumpangkan Rp50.000,00 untuk dijadikan donasi pembagian baju musim dingin di suatu daerah yang membutuhkan.",
"Delapan dari sepuluh pembeli memilih untuk memberikan donasi.",
"Menurut anda mengapa banyak dari pembeli yang memilih berdonasi?"]

In [ ]:
stimulus_a_text = " ".join(stimulus_a)
stimulus_b_text = " ".join(stimulus_b)

for df in [train_a, dev_a, test_a]:
    df["TEXT"] = stimulus_a_text + " [SEP] " + df["RESPONSE"]

for df in [train_b, dev_b, test_b]:
    df["TEXT"] = stimulus_b_text + " [SEP] " + df["RESPONSE"]


In [ ]:
print(train_a.info())
print(train_a.describe(include='all'))  
print(train_b.info())
print(train_b.describe(include='all'))  

In [ ]:
sns.countplot(x='LABEL', data=train_a)
plt.title('Distribusi Label (Data_A Train)')
plt.show()

sns.countplot(x='LABEL', data=train_b)
plt.title('Distribusi Label (Data_B Train)')
plt.show()

---


# Preprocess


In [ ]:
stopwords_ukara = {'yang', 'lebih', 'untuk', 'akan', 'mereka', 'dan'}

In [ ]:
def preprocess(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stopwords_ukara]
    return " ".join(tokens)


In [ ]:
train_a["clean_response"] = train_a["RESPONSE"].apply(preprocess)
train_b["clean_response"] = train_b["RESPONSE"].apply(preprocess)
test_a["clean_response"] = test_a["RESPONSE"].apply(preprocess)
test_b["clean_response"] = test_b["RESPONSE"].apply(preprocess)
dev_a["clean_response"] = dev_a["RESPONSE"].apply(preprocess)
dev_b["clean_response"] = dev_b["RESPONSE"].apply(preprocess)


---


## Embedding


In [ ]:
try:
    sbert_model
except NameError:
    sbert_model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')


In [ ]:
# try:
#     sbert_model
# except NameError:
#     sbert_model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

---


In [ ]:
def extract_features(df, sbert_model):
    embeddings = sbert_model.encode(df['clean_response'].tolist())
    labels = df['LABEL'].values
    return embeddings, labels


In [ ]:
train_embeddings_a, train_labels_a = extract_features(train_a, sbert_model)
dev_embeddings_a, dev_labels_a = extract_features(dev_a, sbert_model)
test_embeddings_a, test_labels_a = extract_features(test_a, sbert_model)

train_embeddings_b, train_labels_b = extract_features(train_b, sbert_model)
dev_embeddings_b, dev_labels_b = extract_features(dev_b, sbert_model)
test_embeddings_b, test_labels_b = extract_features(test_b, sbert_model)



In [ ]:
train_embeddings_a

In [ ]:
train_embeddings_b

---


# Model


In [ ]:
class AESModel(nn.Module):
    def __init__(self, input_dim):
        super(AESModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.dropout = nn.Dropout(0.4)
        self.fc2 = nn.Linear(64, 2)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
def train_model_with_params(params, train_embeddings, train_labels):
    if params['use_smote']:
        smote = SMOTE()
        X_train, y_train = smote.fit_resample(train_embeddings, train_labels)
    else:
        X_train, y_train = train_embeddings, train_labels
    
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
    
    model = AESModel(input_dim=X_train.shape[1])
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-4)
    
    current_batch_size = params['batch_size']
    for epoch in range(params['epochs']):
        if params['batch_increase'] > 0 and (epoch+1) % params['increase_freq'] == 0:
            current_batch_size = min(current_batch_size + params['batch_increase'], len(train_dataset))
            
        train_loader = DataLoader(train_dataset, batch_size=current_batch_size, shuffle=True)
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
    return model

In [ ]:
def objective(trial, train_embeddings, train_labels):
    params = {
        'epochs': trial.suggest_int('epochs', 10, 55),
        'batch_size': trial.suggest_int('batch_size', 2, 8),
        'batch_increase': trial.suggest_int('batch_increase', 0, 4),
        'increase_freq': trial.suggest_int('increase_freq', 2, 4),
        'use_smote': trial.suggest_categorical('use_smote', [True, False])
    }
    
    kf = KFold(n_splits=5)
    scores = []
    
    for train_idx, val_idx in kf.split(train_embeddings):
        X_train, X_val = train_embeddings[train_idx], train_embeddings[val_idx]
        y_train, y_val = train_labels[train_idx], train_labels[val_idx]
        
        model = train_model_with_params(params, X_train, y_train)
        
        val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))
        val_loader = DataLoader(val_dataset, batch_size=32)
        
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        scores.append(correct / total)
    
    return np.mean(scores)

In [ ]:
def evaluate_model(model, embeddings, labels):
    dataset = TensorDataset(torch.FloatTensor(embeddings), torch.LongTensor(labels))
    loader = DataLoader(dataset, batch_size=32)
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())
    
    f1 = f1_score(all_labels, all_preds)
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=['Incorrect', 'Correct']))
    
    return {
        'f1': f1,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall
    }

---


## Data A


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, train_embeddings_a, train_labels_a), n_trials=50)

In [ ]:
best_params = study.best_params
print("Best hyperparameters:", best_params)


In [ ]:
final_model = train_model_with_params(best_params, train_embeddings_a, train_labels_a)

In [ ]:
print("\nEvaluation on Validation Set:")
val_metrics = evaluate_model(final_model, dev_embeddings_a, dev_labels_a)

print("\nEvaluation on Test Set:")
test_metrics = evaluate_model(final_model, test_embeddings_a, test_labels_a)

In [ ]:
print("\nFinal Metrics:")
print(f"Validation - Accuracy: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1']:.4f}")
print(f"Test - Accuracy: {test_metrics['accuracy']:.4f}, F1: {test_metrics['f1']:.4f}")

---


## Data B


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, train_embeddings_b, train_labels_b), n_trials=50)

In [ ]:
best_params = study.best_params
print("Best hyperparameters:", best_params)


In [ ]:
final_model = train_model_with_params(best_params, train_embeddings_b, train_labels_b)

In [ ]:
print("\nEvaluation on Validation Set:")
val_metrics = evaluate_model(final_model, dev_embeddings_b, dev_labels_b)

print("\nEvaluation on Test Set:")
test_metrics = evaluate_model(final_model, test_embeddings_b, test_labels_b)

In [ ]:
print("\nFinal Metrics:")
print(f"Validation - Accuracy: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1']:.4f}")
print(f"Test - Accuracy: {test_metrics['accuracy']:.4f}, F1: {test_metrics['f1']:.4f}")

---


In [ ]:
def analyze_errors(model, embeddings, labels, texts):
    dataset = TensorDataset(torch.FloatTensor(embeddings), torch.LongTensor(labels))
    loader = DataLoader(dataset, batch_size=32)
    
    wrong_samples = []
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(loader):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            for j in range(len(labels)):
                if preds[j] != labels[j]:
                    wrong_samples.append({
                        'text': texts[i*32 + j],
                        'true': labels[j].item(),
                        'predicted': preds[j].item()
                    })
    
    print(f"Total wrong predictions: {len(wrong_samples)}")
    for sample in wrong_samples[:10]:  # Cetak 10 contoh pertama
        print(f"Text: {sample['text']}")
        print(f"True: {sample['true']}, Predicted: {sample['predicted']}\n")
    
    return wrong_samples

wrong_samples = analyze_errors(final_model, test_embeddings_b, test_labels_b, test_b['RESPONSE'].tolist())
wrong_samples

# Opsi

1. Bert prepocessing + Model
2. Bert Prepocessing, Model Neural
3. Bert ...
